In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [8]:
df = pd.read_csv('streamer_df')

In [158]:
def get_data(url, pages):
    import time
    t=5
    df = parse_html(f'{url}=1') # getting the first page
    pages = range(2,pages)
    for page in pages: # getting all other pages, defined by argument
        print(page)
        new_df = parse_html(f"{url}{page}")
        df = df.append(new_df, ignore_index=True)
        time.sleep(t)
    return df

def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

# creates a dataframe for social media plattforms scraped from twitch for a given df with usernames in it
def get_social(df):
    ''' this function scrapes the URL to youtube and twitter for a username-list'''
    import time
    t=5
    usernames = df.username.to_list()
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    chrome_driver_binary = "/Users/home/Documents/Bootcamp/chromedriver"
    dict = {}
    for user in usernames:
        URL= f'https://www.twitch.tv/{user}/about'
        driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
        driver.get(URL)
        soup=BeautifulSoup(driver.page_source)
        twitter = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "twitter.com/" in a["href"]]))
        youtube = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "youtube.com/" in a["href"]]))
        instagram = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "instagram" in a["href"]]))
        discord = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "discord" in a["href"]]))
        if twitter:
            twitter = twitter[0]
        if youtube:
            youtube = youtube[0]
        if instagram:
            instagram = instagram[0]
        if discord:
            discord = discord[0]
        dict[user] = [twitter, youtube, instagram, discord]
        time.sleep(t)
    df = pd.DataFrame.from_dict(dict, orient='index')
    df['Twitter'] = df[0].apply(lambda x: None if x == [] else x)
    df['YouTube'] = df[1].apply(lambda x: None if x == [] else x)
    df['Instagram'] = df[2].apply(lambda x: None if x == [] else x)
    df['Discord'] = df[3].apply(lambda x: None if x == [] else x)
    df.index.name = 'username'
    df = df.reset_index()
    return df[['username', 'Twitter', 'YouTube', 'Instagram', 'Discord']]

# transfer stuff to csv
# df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/put_name', index=False)

In [159]:
df_new = get_social(df.sample(2))

<ipython-input-158-3bedb9b58feb>:32: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


In [161]:
df_new

,username,Twitter,YouTube,Instagram,Discord
0,blncgjk,None,None,None,None
1,D3stri,None,https://www.youtube.com/channel/ucypaungxezct7...,None,https://discord.gg/tgkmygg


In [135]:
df_new2.iloc[0,1] == []

True

In [110]:
# df_new2['Twitter'] = df_new2[0].str.contains('twitter.')
df_new2['Twitter'] = df_new2[0][df_new2[0].str.contains('twitter.com')]

In [52]:
df_new2["Twitter"] = df_new2[0].apply(lambda x: x if 'twitter' in x else False)

In [4]:
url = 'https://twitchtracker.com/statistics/games'

In [5]:
test = parse_html(url)

ValueError: No tables found matching pattern '.+'